# Web Scrape labelled images of building styles

http://www.acotoronto.ca/tobuilt_new_detailed.php

American colonial
Annex Style
Art deco
Arts and Crafts
Beaux arts
Brutalist
Byzantine
Chicago style
Classical revival
Commercial style
Contemporary
Deconstructivism
Dutch colonial
Early modern
Edwardian classical
English Cottage style
Georgian revival
Gothic revival
Greek revival
International style
Italianate
Late modernist
Log construction
Mid century expressionist
Mirrored tower
Modern classical
Modern historicist
Modernist
Neo Palladian
Neo-Chateau
Neo-Georgian
Neo-modernist
Neo-Tudor
Postmodern
Prairie style
Queen Anne
Regency
Renaissance revival
Richardsonian Romanesque
Romanesque revival
Sculptural
Second empire
Shingle style
Spanish colonial
Toronto Bay and Gable
Workers Cottage

In [12]:
# Import libaries
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import string
import re
import urllib
import os
import time
from datetime import datetime

Load Page

In [13]:
def load_page(url):
    response = requests.get(url)
    if response.status_code == 200:
        return response.text
    else:
        print(f"Error connecting: status code {response.status_code}")

In [3]:
# def load_page(this_url):
    
   
#     # Establish the connection to the web page:
#     response = requests.get(this_url)
#     if response.status_code != 200:
#         print( "Error ")

#     # You can use status codes to understand how the target server responds to your request.
#     # Ex., 200 = OK, 400 = Bad Request, 403 = Forbidden, 404 = Not Found.
#     print('Status Code: ',response.status_code)

#     # Pull the HTML string out of requests and convert it to a Python string.
#     html = response.text
#     if debug==True:
#         # The first 700 characters of the content.
#         print("\nFirst part of HTML document fetched as string:\n")
#         print(html[:2000])
#     return html

# # have to read Location from html

In [26]:
def download_image(style, img_url):
    '''
    * downloads image to the appropriate style sub-folder in the images directory (and creates folder if missing)
    * to test: download_image('test','/tobuilt_bk/php/Buildingimages/106BedfordRd.jpg')
    '''

    dest_dir = f"../Images/{style}"

    if not os.path.exists(dest_dir):
        os.makedirs(dest_dir)
    full_url = f'{site_root}/{img_url}'
    image_path = f"{dest_dir}/{img_url.split('/')[-1]}"
#     print(image_path)
#     print(full_url)
    res = urllib.request.urlretrieve(full_url, image_path)
    #print(res)

#### Building Details pages
* page is structured with alternating building_info and building_info2 divs
* building_info contains the name of the info to follow
* building_info2 contains the value
* The companies row contains the architects.  There can be muliple architects for a building, so these are < li > items.
* Sometimes the archtect is a hyperlink, but not always, so need to handle both cases specially

In [27]:
def get_building_details(bld_link):
    
    flags_dict = {
        'Completed':None,
        'Demolished': None,
        'Companies': None,
        'Type':None,
        'Current use': None,
        'Heritage': None,
       'Notes': None
    }
    architects=[]
    
    full_url = f'{site_root}{bld_link}'
    #print(full_url)
    html_bld = load_page(full_url)
    soup_bld = BeautifulSoup(html_bld)
    # get all building info elements
    info = soup_bld.find_all('div',{'class': 'building_info'})
    for inf in info:
        for flag in flags_dict.keys():
            if flag in inf.text:
                if 'Companies' in flag:
                    company_box = inf.find_next()

                    if 'Architect - ' in company_box.text:
                        li_items = company_box.find_all('li')

                        for arc in li_items:
                            if (arc.find('span')):
                                architect = arc.find('span').previousSibling
                                if architect.name =='a':
                                    architects.append(architect.text.strip())
                                else:
                                    architects.append(architect.replace('-','').strip())
                                
                        flags_dict[flag] = architects
                else:
                    flags_dict[flag] = inf.find_next().text.strip()
    return flags_dict

### Get Buildings for style

In [28]:
def get_buildings_for_style(soup, style):
    buildings=[]
   
    for building in soup.find_all('div',{'class': 'box'}):
        build_style = building.find('div',{'class': 'box_image'})['style']
        image_url = re.findall('\((.*?)\)', build_style)[0]
        build_dict={
            'Style':style,
            'Name':building.findChild('span',{'class':'title'}).text,
            'Bld_link':building.findChild('a').get_attribute_list(key='href')[0],
            'Image':image_url,
            'Address': str(building.findChild('div',{'class':'the_box_text'}).findChild('p')).replace('<br/>',' ').replace('<p>','').replace('</p>','')
        }

        download_image(style,image_url)
        #follow link to get more info on building
        if debug==True: 
            print(build_dict['Bld_link'])
        build_details_dict = get_building_details(build_dict['Bld_link'])
        build_dict = {**build_dict, **build_details_dict}
        buildings.append(build_dict)
   # bld_df = pd.DataFrame(buildings)
    return buildings

### Set up base variables

In [29]:
#styles=['Annex%20Style','Arts%20and%20Crafts']
#base_url = "http://www.acotoronto.ca/search_buildingsR-d.php?sid=7832"
main_page = 'http://www.acotoronto.ca/tobuilt_new_detailed.php'
style_url="http://www.acotoronto.ca/search_buildingsDB_d2.php"
site_root = "http://www.acotoronto.ca/"
debug=False
buildings_list=[]
# df_columns = ['Companies', 'Completed', 'Current use', 'Demolished', 'Heritage',
#        'Notes', 'Type', 'address', 'bld_link', 'image', 'name', 'style']

In [31]:

html = load_page(main_page)
soup = BeautifulSoup(html)
style_options = soup.find('select',{'name':'MainStyle'}).find_all('option')
for style in style_options[1:]:
    print(style.text)
process_style(style_options[1].text)    


American colonial
Annex Style
Art deco
Arts and Crafts
Beaux arts
Brutalist
Byzantine
Chicago style
Classical revival
Commercial style
Contemporary
Deconstructivism
Dutch colonial
Early modern
Edwardian classical
English Cottage style
Georgian revival
Gothic revival
Greek revival
International style
Italianate
Late modernist
Log construction
Mid century expressionist
Mirrored tower
Modern classical
Modern historicist
Modernist
Neo Palladian
Neo-Chateau
Neo-Georgian
Neo-modernist
Neo-Tudor
Postmodern
Prairie style
Queen Anne
Regency
Renaissance revival
Richardsonian Romanesque
Romanesque revival
Sculptural
Second empire
Shingle style
Spanish colonial
Toronto Bay and Gable
Workers Cottage
loading American colonial
[{'Style': 'American colonial', 'Name': 'Robert Grieg House', 'Bld_link': 'show_building.php?BuildingID=4386', 'Image': '/tobuilt_bk/php/Buildingimages/20ChestnutPark.jpg', 'Address': '20 Chestnut Park Rosedale Toronto', 'Completed': '1905', 'Demolished': None, 'Companies': ['A

http://www.acotoronto.ca/search_buildingsDB_d2.php?MainStyle=Postmodern
http://www.acotoronto.ca/search_buildingsDB_d2.php?MainStyle=Arts%20and%20Crafts
http://www.acotoronto.ca/search_buildingsDB_d2.php?MainStyle=Annex%20Style

In [24]:
#building_df=pd.DataFrame(columns=df_columns)
#for style in styles:
def process_style (style):
    print (f"loading {style}")
    curr_style_url = f"{style_url}?MainStyle={style}"
    #print(curr_style_url)
    html = load_page(curr_style_url)
    soup = BeautifulSoup(html)
    if 'document.location' in soup.text:
        # need to redirect to another page
        redirect_url = soup.text.strip().replace('document.location = "','').replace('";',"")
        curr_style_url=f"{site_root}{redirect_url}"
       # print(curr_style_url)
        html2 = load_page(curr_style_url)
        soup = BeautifulSoup(html2)
        
    style_list = get_buildings_for_style(soup, style)
    print(style_list)
    buildings_list.extend(style_list)
#     else:
#         style_df=get_buildings_for_style(soup, style)
#     building_df=pd.concat([building_df,style_df],axis=0)
   # building_df.join.merge(style_df)
#soup.text
#http://www.acotoronto.ca/search_buildingsR-d.php?sid=8023

In [32]:
buildings_list
#bld_df = pd.DataFrame(buildings_list)

[{'Style': 'American colonial',
  'Name': 'Robert Grieg House',
  'Bld_link': 'show_building.php?BuildingID=4386',
  'Image': '/tobuilt_bk/php/Buildingimages/20ChestnutPark.jpg',
  'Address': '20 Chestnut Park Rosedale Toronto',
  'Completed': '1905',
  'Demolished': None,
  'Companies': ['Alfred E. Boultbee'],
  'Type': 'Detached house',
  'Current use': 'Residential',
  'Heritage': 'South Rosedale Heritage Conservation District',
  'Notes': 'First Occupant: Greig, Robert\r\n\r\nFirst OccupantÂ´s Occupation: Pres The Robert Greig Co. Ltd.'},
 {'Style': 'American colonial',
  'Name': '22 Chestnut Park',
  'Bld_link': 'show_building.php?BuildingID=4388',
  'Image': '/tobuilt_bk/php/Buildingimages/22ChestnutPark.jpg',
  'Address': '22 Chestnut Park Rosedale Toronto',
  'Completed': '1905',
  'Demolished': None,
  'Companies': ['Alfred E. Boultbee'],
  'Type': 'Detached house',
  'Current use': 'Residential',
  'Heritage': 'South Rosedale Heritage Conservation District',
  'Notes': 'First

In [33]:
bld_df = pd.DataFrame(buildings_list)

In [34]:
bld_df

,Address,Bld_link,Companies,Completed,Current use,Demolished,Heritage,Image,Name,Notes,Style,Type
0,20 Chestnut Park Rosedale Toronto,show_building.php?BuildingID=4386,[Alfred E. Boultbee],1905,Residential,None,South Rosedale Heritage Conservation District,/tobuilt_bk/php/Buildingimages/20ChestnutPark.jpg,Robert Grieg House,"First Occupant: Greig, Robert\r\n\r\nFirst Occ...",American colonial,Detached house
1,22 Chestnut Park Rosedale Toronto,show_building.php?BuildingID=4388,[Alfred E. Boultbee],1905,Residential,None,South Rosedale Heritage Conservation District,/tobuilt_bk/php/Buildingimages/22ChestnutPark.jpg,22 Chestnut Park,"First Occupant: Falconbridge, John D.\r\n\r\nF...",American colonial,Detached house


In [35]:
bld_df.to_csv('../data/aco_buildings_'+ str(round(time.time(),0)) + '.csv')

In [ ]:
# blg =soup.find('div',{'class': 'box'})
# bld_link = blg.findChild('a').get_attribute_list(key='href')[0]
# style = blg.find('div',{'class': 'box_image'})['style']
# url = re.findall('\((.*?)\)', style)[0]
# name=blg.findChild('span',{'class':'title'}).text
# address=str(blg.findChild('div',{'class':'the_box_text'}).findChild('p')).replace('<br/>',' ').replace('<p>','').replace('</p>','')

In [ ]:
# def get_buildings_for_style(soup, style):
#     buildings=[]
#     for building in soup.find_all('div',{'class': 'box'}):
#         style = building.find('div',{'class': 'box_image'})['style']
#         image_url = re.findall('\((.*?)\)', style)[0]
#         build_dict={

#             'name':building.findChild('span',{'class':'title'}).text,
#             'bld_link':building.findChild('a').get_attribute_list(key='href')[0],
#             'image':image_url,
#             'address': str(building.findChild('div',{'class':'the_box_text'}).findChild('p')).replace('<br/>',' ').replace('<p>','').replace('</p>','')
#         }

#         buildings.append(build_dict)
#         download_image(style,image_url)
#         #follow link to get more info on building
#         print(get_building_details(build_dict['bld_link']))
#     bld_df = pd.DataFrame(buildings)

In [ ]:
# # following building link
# style="Arts%20and%20Crafts"
# bld_link = "show_building.php?BuildingID=3256"
# #curr_style_url = f"{style_url}?MainStyle={style}"
# full_url = f'{site_root}{bld_link}'
# print(full_url)
# html_bld = load_page(full_url)
# soup_bld = BeautifulSoup(html_bld)
# #soup_bld.text
# #info = soup_bld.find_all('div',{'class': 'building_info2'})
# info = soup_bld.find_all('div',{'class': ['building_info','building_info2']})
# get_comp_date_flg=False
# get_demol_date_flg = False
# ix=0


# # reset all variables
# completed_date=None
# demolished_date = None
# bld_type = None
# bld_curr_use = None
# bld_heritage = None
# is_demolished = False
# bld_status = None
# architects = []

# for inf in info:
#     print("********")
#     print(inf.text.strip())
    
#     if get_comp_date_flg:
#         completed_date = inf.text.strip()
#         get_comp_date_flg = False
#     elif get_demol_date_flg:
#         demolished_date = inf.text.strip()
#         get_demol_date_flg = False 
        
#     # warning that next row will contain the actual date
#     if 'Completed' in inf.text:
#         get_comp_date_flg = True
#         bld_status= 'Completed'
#     elif 'Demolished' in inf.text:
#         get_demol_date_flg = True
#         bld_status='Demolished'
#     if 'Architect - ' in inf.text:
#         li_items = inf.find_all('li')
        
#         for arc in li_items:
#             #print(arc)
#             #architect = arc.text.strip()
#             architect = arc.find('span').previousSibling
#             if architect.name =='a':
#                 #architect = architect.text
#                 architects.append(architect.text.strip())
#             else:
#                 architects.append(architect.replace('-','').strip())
#             #print(architect)
#         #print("***")
#         # note: this is not consistent -- only sometimes is architect a link
#         # stored as a list item (example building with 2 architects = 3256)
#     # #5 6 and 7 are type, current use and heritage?
#     if ix==5:
#         bld_type = inf.text.strip()
#     elif ix==6:
#         bld_curr_use = inf.text.strip()
#     elif ix==7:
#         bld_heritage = inf.text.strip()
#     #TODO: get Notes: 
#     ix=ix+1
    

# print(completed_date)
# print(demolished_date)
# print(bld_status)
# #print(architect)#.split('\n')[2]

# print(architects)
# #print(re.findall("(?=\-)(.*?)(?=\<)", architect)[0].replace('-','').strip())

# print(bld_type)
# print(bld_curr_use)
# print(bld_heritage)

In [ ]:
# # following building link
# style="Arts%20and%20Crafts"
# bld_link = "show_building.php?BuildingID=3256"
# #curr_style_url = f"{style_url}?MainStyle={style}"
# full_url = f'{site_root}{bld_link}'
# print(full_url)
# html_bld = load_page(full_url)
# soup_bld = BeautifulSoup(html_bld)
# #soup_bld.text
# #info = soup_bld.find_all('div',{'class': 'building_info2'})
# info = soup_bld.find_all('div',{'class': ['building_info','building_info2']})
# get_comp_date_flg=False
# get_demol_date_flg = False
# ix=0


# # reset all variables
# completed_date=None
# demolished_date = None
# bld_type = None
# bld_curr_use = None
# bld_heritage = None
# is_demolished = False
# bld_status = None
# architects = []
# # Heritage matches on heritage and Heritage Property value
# flags_list =['Completed','Demolished','Type','Current use', 'Heritage']
# #vars_list = [completed_date,demolished_date]
# flags_dict = {
#     'Completed':None,
#     'Demolished': None,
#     'Architects': None,
#     'Type':None,
#     'Current use': None,
#     'Heritage': None
    
# }
# flag_thrown = False
# flag_value = None

# for inf in info:
#     print("********")
#     print(inf.text.strip())
    
#     if flag_thrown == True:
#         flags_dict[flag_value] = inf.text.strip()
#         flag_thrown=False
#         #print(flags_dict[flag_value])
#     for flag in flags_dict.keys():
#         if flag in inf.text and 'Heritage property' not in inf.text:
#             flag_thrown = True
#             flag_value = flag
#             print (f'throwing flag {flag} for {inf.text.strip()}')
            
#     if 'Architect - ' in inf.text:
#         li_items = inf.find_all('li')
        
#         for arc in li_items:
#             #print(arc)
#             #architect = arc.text.strip()
#             architect = arc.find('span').previousSibling
#             if architect.name =='a':
#                 #architect = architect.text
#                 architects.append(architect.text.strip())
#             else:
#                 architects.append(architect.replace('-','').strip())
# print (flags_dict)
# #     if get_comp_date_flg:
# #         completed_date = inf.text.strip()
# #         get_comp_date_flg = False
# #     elif get_demol_date_flg:
# #         demolished_date = inf.text.strip()
# #         get_demol_date_flg = False 
        
# #     # warning that next row will contain the actual date
# #     if 'Completed' in inf.text:
# #         get_comp_date_flg = True
# #         bld_status= 'Completed'
# #     elif 'Demolished' in inf.text:
# #         get_demol_date_flg = True
# #         bld_status='Demolished'
# #     if 'Architect - ' in inf.text:
# #         li_items = inf.find_all('li')
        
# #         for arc in li_items:
# #             #print(arc)
# #             #architect = arc.text.strip()
# #             architect = arc.find('span').previousSibling
# #             if architect.name =='a':
# #                 #architect = architect.text
# #                 architects.append(architect.text.strip())
# #             else:
# #                 architects.append(architect.replace('-','').strip())
# #             #print(architect)
# #         #print("***")
# #         # note: this is not consistent -- only sometimes is architect a link
# #         # stored as a list item (example building with 2 architects = 3256)
# #     # #5 6 and 7 are type, current use and heritage?
# #     if ix==5:
# #         bld_type = inf.text.strip()
# #     elif ix==6:
# #         bld_curr_use = inf.text.strip()
# #     elif ix==7:
# #         bld_heritage = inf.text.strip()
# #     #TODO: get Notes: 
# #     ix=ix+1
    

# # print(completed_date)
# # print(demolished_date)
# # print(bld_status)
# # #print(architect)#.split('\n')[2]

# # print(architects)
# # #print(re.findall("(?=\-)(.*?)(?=\<)", architect)[0].replace('-','').strip())

# # print(bld_type)
# # print(bld_curr_use)
# # print(bld_heritage)

In [ ]:
# # following building link
# style="Arts%20and%20Crafts"
# bld_link = "show_building.php?BuildingID=3458"
# #curr_style_url = f"{style_url}?MainStyle={style}"
# full_url = f'{site_root}{bld_link}'
# print(full_url)
# html_bld = load_page(full_url)
# soup_bld = BeautifulSoup(html_bld)
# #soup_bld.text
# #info = soup_bld.find_all('div',{'class': 'building_info2'})
# info = soup_bld.find_all('div',{'class': 'building_info'})
# get_comp_date_flg=False
# get_demol_date_flg = False
# ix=0


# # reset all variables

# architects = []
# # Heritage matches on heritage and Heritage Property value
# #flags_list =['Completed','Demolished','Companies','Type','Current use', 'Heritage']
# #vars_list = [completed_date,demolished_date]
# flags_dict = {
#     'Completed':None,
#     'Demolished': None,
#     'Companies': None,
#     #'Architects': None,
#     'Type':None,
#     'Current use': None,
#     'Heritage': None,
#    'Notes': None
# }
# flag_thrown = False
# flag_value = None

# for inf in info:
#     for flag in flags_dict.keys():
#         if flag in inf.text:
#             if 'Companies' in flag:
#                 company_box = inf.find_next()

#                 if 'Architect - ' in company_box.text:
#                     li_items = company_box.find_all('li')

#                     for arc in li_items:
#                         architect = arc.find('span').previousSibling
#                         if architect.name =='a':
#                             architects.append(architect.text.strip())
#                         else:
#                             architects.append(architect.replace('-','').strip())
#                     flags_dict[flag] = architects
#             else:
#                 flags_dict[flag] = inf.find_next().text.strip()
# flags_dict

In [ ]:
#